In [1]:
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go
#     import plotly
    import pickle,random
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    user_df=user_df.loc[user_df["user_generated"]==0]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/day_hour_minutes.pickle', 'rb') as handle:
        watch_worn = pickle.load(handle)
    
    if user_id in watch_worn:
        users_ppg_mins=watch_worn[user_id]
    if len(user_df)>0:

#         user_df_not_curr_week=user_df.loc[user_df["week"] != week_num]
        user_df["start_hour"]=user_df["starttime"].dt.hour
        user_df["end_hour"]=user_df["endtime"].dt.hour
        user_df_top_stressors=user_df.groupby(["Stressor"],as_index=False)["counter"].count().sort_values("counter",ascending=False).head(5).Stressor.values.tolist()
        user_df=user_df.loc[user_df["Stressor"].isin(user_df_top_stressors)]
        

        stressors_data =pd.DataFrame([])

        for index, row in user_df.iterrows():
            if row["start_hour"] == row["end_hour"]:
                stressors_data = stressors_data.append(pd.DataFrame({'Stressor': row["Stressor"],'Start Date': row["episode_start_day"],'end Date': row["episode_end_day"],'start_hour':row["starttime"],'end_hour':row["endtime"],
                                                                         'duration':row["duration"],'week':row["week"],
                                                                       'Total ppg minutes':users_ppg_mins[row["episode_start_day"]][row["start_hour"]],

                                                                      }, 
                                                                      index=[0]), ignore_index=True)

            elif row["start_hour"] != row["end_hour"] :
                    if row["episode_start_day"] == row["episode_end_day"]:
                        total_ppg_minutes=users_ppg_mins[row["episode_start_day"]][row["start_hour"]]\
                                            +users_ppg_mins[row["episode_end_day"]][row["end_hour"]]+ 60*(row["end_hour"] -row["start_hour"]-1)
                        stressors_data = stressors_data.append(pd.DataFrame({'Stressor': row["Stressor"],'Start Date': row["episode_start_day"],'end Date': row["episode_end_day"],
                                                                             'start_hour':row["starttime"],'end_hour':row["endtime"],
                                                                                      'duration':row["duration"],'week':row["week"],
                                                                               'Total ppg minutes':total_ppg_minutes,

                                                                              }, 
                                                                              index=[0]), ignore_index=True)
                    else:

                        total_ppg_minutes=users_ppg_mins[row["episode_start_day"]][row["start_hour"]] \
                                            +users_ppg_mins[row["episode_end_day"]][row["end_hour"]]+ 60*(row["end_hour"] -row["start_hour"])%24

                        stressors_data = stressors_data.append(pd.DataFrame({'Stressor': row["Stressor"],'Start Date': row["episode_start_day"],'end Date': row["episode_end_day"],'start_hour':row["starttime"],'end_hour':row["endtime"],
                                                                                  'duration':row["duration"],
                                                                               'Total ppg minutes':total_ppg_minutes,

                                                                              }, 
                                                                              index=[0]), ignore_index=True)
    
    
        stressors_data["Duration/mins"]=round(stressors_data['duration']/stressors_data['Total ppg minutes']*60,2)
        stressors_data=stressors_data.groupby(["Stressor","week"],as_index=False)["Duration/mins"].mean()
        stressors_data["Duration/mins"]=round(stressors_data['Duration/mins'],2)
        weeks=stressors_data.week.unique()
        stressors_data["Week"] = stressors_data["week"].astype(str)
        color_discrete_map={}

        for i in range(1,int(week_num)+1):
            if i == int(week_num):
                color_discrete_map[str(i)]="#000000"
            else:
                color_discrete_map[str(i)]="#D55E00"

        stressors_data['Stressor'] = pd.Categorical(
                        stressors_data['Stressor'], 
                        categories=user_df_top_stressors, 
                        ordered=True
                    )

        stressors_data=stressors_data.sort_values("Stressor")
        fig = px.bar(stressors_data, x="Duration/mins", y="week", facet_col="Stressor", orientation='h',color="Week",
                     labels="week",facet_col_wrap=1,facet_row_spacing=0.05,

                      color_discrete_map=color_discrete_map)
#         user_df_not_curr_week=stressors_data.loc[stressors_data["week"] != week_num]
#         user_df_not_curr_week=user_df_not_curr_week.loc[user_df_not_curr_week["Stressor"].isin(user_df_top_stressors)]
#         user_df_not_curr_week["Duration/mins"]=user_df_not_curr_week["Duration/mins"].div(7).round(2)
#         user_df_not_curr_week=user_df_not_curr_week.groupby("Stressor",as_index=True)["Duration/mins"].mean()
#         print(user_df_not_curr_week)
#         print(stressors_data)
#         user_df_not_curr_week["duration"]=user_df_not_curr_week["duration"].div(7).round(2)

#         user_df_not_curr_week=user_df_not_curr_week.to_dict()
#         print(user_df_not_curr_week)
        
#         for i in range(len(user_df_top_stressors)):

#             fig.add_shape(dict(type="line",x0=user_df_not_curr_week[user_df_top_stressors[i]],x1=user_df_not_curr_week[user_df_top_stressors[i]],
#                           y0=0,y1=str(week_num+1),line_color="#0072B2"),row=len(user_df_top_stressors)-i,col=1)

        
        fig.update_layout(barmode='relative')
        fig.update_traces(width=0.5,selector=dict(type='bar'))
        # fig.update_yaxes(matches=None)
        fig.update_yaxes( secondary_y=False)

        fig.update_xaxes(showgrid=True, ticks="inside",range=[1,60],dtick=5)
        fig.update_yaxes(showgrid=True, ticks="inside",range=[0.5,week_num+0.5],nticks=int(week_num/2)+1,title="Week #")
        fig.update_xaxes(matches='x')
        fig.update_yaxes(matches='y')
        fig.update_xaxes( showticklabels=True, visible=True)
        fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Stressor=", "")))

        fig.update_layout(bargap=0.5,showlegend=False,

            xaxis_title="Stressed minutes per hour",
        title_font_family="Times New Roman",
             title_font_color="Black",
            font=dict(
                family="Courier New, monospace",
                size=12,
                color="Black"
            )
        )


        fig.update_layout(
                    autosize=True,

                    margin=dict(
                       l=30,
                            r=20,
                            b=40,
                            t=30,

                    ),)   




        data_summary_user=data_summary_user_current["Stressor Analysis"]
        data_summary_user=data_summary_user.replace("week_num", str(week_num))


        user_df_curr_week=stressors_data.loc[stressors_data["week"] == str(week_num)]
        
        if len (user_df_curr_week) > 0:
            count=1
            temp=["_a_","_b_","_c_","_d_","_e_"]
            for stressor in user_df_top_stressors:
                data_summary_user=data_summary_user.replace("_s"+str(count)+"_", str(stressor))
                
                temp_week=user_df_curr_week.loc[user_df_curr_week["Stressor"]==stressor]["Duration/mins"].values.tolist()
                
                if len(temp_week) > 0:
                    data_summary_user=data_summary_user.replace(temp[count-1], str(temp_week[0]))
                    
                else:
                    data_summary_user=data_summary_user.replace(temp[count-1], str("This stressor didnt occur this week"))
                    data_summary_user=data_summary_user.replace("minutes per hour", str(""))
                count+=1
                
        else:
            data_summary_user="No stressor reported this week " + str(week_num)

        return "Stressor Analysis",fig,data_summary_user
    else:
        return "","",""